In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [4]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [6]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [7]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [9]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
#description_list = get_job_description(result_page)
#empresa_name = get_name_empresa(result_page)
#output = unify_lists([empresa_name, anuncio_list, description_list])

In [12]:
def trabajopolis(job_description):
    import time
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        url = "https://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="
        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [48]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1000"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_link)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [49]:
job_names = ["analista", "desarrollador", "programador", "quimico", "finanzas", "mensajero", "albañil"]
job_names1 = ["Administración", "Oficina", "Atención", "Cliente", "Banca", "Contabilidad"]
job_names2 = ["Economía", "Dirección", "Gerencia", "Diseño", "Marketing", "Ventas"]
job_names3 = ["Vendedor", "Medicina", "Profesor", "Técnico", "Educación", "Legal", "Asesoría", "Turismo"]
job_names4 = ['Personal', 'conocimientos']

Results = trabajopolis2()

In [ ]:
import csv
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/trabajo123.csv", 'a',newline='')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        for cell in Results[i][0]:
            #print(cell)
           csvRow.append(cell)
        csvRow.append(Results[i][1])
        csvRow.append(Results[i][2])
        #print(csvRow)
        writer.writerow(csvRow)
finally:
    outfile.close() 